<a href="https://colab.research.google.com/github/RopakS/insaid/blob/master/RAG/LLM_Openrouter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://openrouter.ai/settings/keys

In [1]:
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 5.1 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.8 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 86.9 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/deepanrajm/deep_learning.git

Cloning into 'deep_learning'...
remote: Enumerating objects: 3144, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 3144 (delta 66), reused 63 (delta 27), pack-reused 3025 (from 2)
Receiving objects: 100% (3144/3144), 309.98 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (306/306), done.
Updating files: 100% (2653/2653), done.


In [6]:
!pip install langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
from langchain_openai import ChatOpenAI # Updated import
from langchain_core.messages import HumanMessage

# Step 1: Set your OpenRouter API key and endpoint
# os.environ["OPENAI_API_KEY"] = "api-key"  # Replace with your actual OpenRouter API key
# os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
#https://openrouter.ai/settings/keys

# Step 1: Set your OpenRouter API key and endpoint
#DeepanLLMKey1
#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

os.environ["OPENAI_API_KEY"] = "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"  # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-70b-instruct:free",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm.invoke([HumanMessage(content=query)]) # Corrected method call
    print("\nAssistant:", response.content)

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader("/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])


In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from copy import deepcopy

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader(r"/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
semantic_chunks = text_splitter.split_documents(docs)

# manually apply overlap across those semantic chunks
def apply_overlap_to_chunks(chunks, overlap_characters=200):
    overlapped_chunks = []

    for i in range(len(chunks)):
        curr = deepcopy(chunks[i])
        if i > 0:
            prev = chunks[i - 1].page_content
            # Get the last N characters of the previous chunk
            overlap = prev[-overlap_characters:]
            # Prepend to current chunk
            curr.page_content = overlap + curr.page_content
        overlapped_chunks.append(curr)

    return overlapped_chunks

# Apply overlap (adjust 200 to suit your context)
documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])
